
# WEB Scraping para buscar coche de segunda mano

A lo largo de este ejercicio vamos a realizar Web Scraping de la página https://www.coches.com/coches-segunda-mano/coches-ocasion.htm para recoger los datos de todos los coches usados que existen a la venta y poder hacer un análisis posterior.
  
Para ello vamos a necesitar las siguientes librerías:

In [1]:
import requests
from bs4 import BeautifulSoup
import csv
import re
import pandas as pd

Primeramente hemos analizado el archivo robots.txt y se ha visto que no tenemos restricción alguna para hacer Web Scraping con Python. Luego se ha analizado la estructura de la web para ver como realizar la extracción de datos; se hará en la sección de coches de segunda mano y por páginas (estableceremos un límite de páginas para no saturar). Finalmente se ha inspeccionado el tamaño de la web y han salido 3800 resultados.

A continuación con las funciones "builtwith" y "whois" de python comprobaremos la tecnología de la página web y su propietario.

In [2]:
import builtwith

builtwith.builtwith("https://www.coches.com/coches-segunda-mano/coches-ocasion.htm")

{'tag-managers': ['Google Tag Manager'],
 'javascript-frameworks': ['Prototype', 'RequireJS', 'jQuery'],
 'web-frameworks': ['Twitter Bootstrap']}

In [3]:
import whois

print(whois.whois("https://www.coches.com/coches-segunda-mano/coches-ocasion.htm"))

{
  "domain_name": [
    "COCHES.COM",
    "coches.com"
  ],
  "registrar": "Dinahosting s.l.",
  "whois_server": "whois.dinahosting.com",
  "referral_url": null,
  "updated_date": [
    "2017-05-29 15:16:40",
    "2017-05-29 17:16:40"
  ],
  "creation_date": "1996-12-07 05:00:00",
  "expiration_date": "2019-12-06 05:00:00",
  "name_servers": [
    "BAYAN.NS.CLOUDFLARE.COM",
    "CHERYL.NS.CLOUDFLARE.COM",
    "bayan.ns.cloudflare.com",
    "cheryl.ns.cloudflare.com"
  ],
  "status": [
    "clientDeleteProhibited https://icann.org/epp#clientDeleteProhibited",
    "clientTransferProhibited https://icann.org/epp#clientTransferProhibited",
    "clientDeleteProhibited (http://www.icann.org/epp#clientDeleteProhibited)",
    "clientTransferProhibited (http://www.icann.org/epp#clientTransferProhibited)"
  ],
  "emails": "abuse-domains@dinahosting.com",
  "dnssec": "unsigned",
  "name": "Redacted by Privacy",
  "org": "Autodescuento SL",
  "address": "Redacted by Privacy",
  "city": "Redacted 

Una vez hecha la exploración inicial, procedemos a extraer los datos

In [4]:
#primero definimos la url y el número máximo de páginas que vamos a explorar
URL_BASE = "https://www.coches.com/coches-segunda-mano/coches-ocasion.htm"
MAX_PAGES = 50

#creamos el archivo csv donde se guardarán los datos extraidos, y añadimos la cabecera.
file_name = "coches_segunda_mano.csv"
f = csv.writer(open(file_name, 'w', newline=''))
f.writerow(['Coche', 'Ubicacion', 'Combustible', 'CV', 'Anno', 'Kilometros', 'Precio'])

#con un bucle for recorremos todas las páginas hasta el máximo prefijado
for i in range(1,MAX_PAGES):
    
    #construcción de la URL
    if i < 1:
        url = URL_BASE + "?page=%d" %(i)
    else:
        url = URL_BASE
        
    #realizamos la petición a la web
    req = requests.get(url)
    # comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = req.status_code
    if statusCode == 200:
        
        #pasamos el contenido HTML de la web a un objeto BeautifulSoup()
        html = BeautifulSoup(req.content, "html.parser")
        
        #obtenemos todas las entradas divs de los coches
        entradas = html.find_all('div', class_=re.compile("pill script__pill car"))
        
        #recorremos todas las entradas para extraer los datos
        for entrada in entradas:
            
            coche = entrada.find('strong', {'class': 'make-model-version'}).getText().strip()
            ubicacion = entrada.find('span', {'class': 'location'}).getText().strip()
            combustible = entrada.find('span', {'class': 'gas'}).getText().strip()
            cv = entrada.find('span', {'class': 'cv'}).getText().strip()
            año = entrada.find('span', {'class': 'year'}).getText().strip()
            km = entrada.find('span', {'class': 'km'}).getText().strip()
            precio = entrada.find('strong', {'class': 'price'}).getText().strip()
            
            km_final = km[:-3]
            precio_final = precio[:-2]
            cv_final = cv[:-3]
            
            #escribimos cada coche en su fila correspondiente del csv
            f.writerow([coche, ubicacion, combustible, cv_final, año, km_final, precio_final])
            
    else:
        print("Ha habido un error en la petición a la página")
        break

In [7]:
# cargamos el archivo csv y vemos que se muestra correctamente
coches = pd.read_csv(u'coches_segunda_mano.csv')
coches

,Coche,Ubicacion,Combustible,CV,Anno,Kilometros,Precio
0,Fiat 500X 1.3Mjt Pop Star,Madrid,Diesel,95,2017,51.142,10.990
1,Mazda Mazda6 2.2DE Style,Madrid,Diesel,150,2015,131.000,12.500
2,Bmw Serie 7 750iA,Madrid,Gasolina,450,2014,37.000,46.900
3,Mercedes Vito Furgon 111CDI,"Rioja, La",Diesel,114,2017,77.490,14.800
4,Bmw X3 sDrive 18d,Madrid,Diesel,150,2016,78.000,22.250
...,...,...,...,...,...,...,...
975,Nissan Qashqai 1.5dCi Tekna,Barcelona,Diesel,110,2017,29.638,15.985
976,Nissan Qashqai 1.5dCi Tekna,Barcelona,Diesel,110,2017,29.638,15.985
977,Toyota Auris 1.6 VVT-i Luna,Barcelona,Gasolina,124,2007,140.000,4.200
978,Bmw X3 xDrive 20dA,Madrid,Diesel,190,2015,85.000,23.750
